In [1]:
 from google.colab import files
uploaded = files.upload()

Saving Structured data.csv to Structured data.csv


In [88]:
import io
import pandas as pd

df = pd.read_csv(io.BytesIO(uploaded['Structured data.csv']))
del df['Unnamed: 0']
df.head()
 

,Facility ID,Zip_Code,Population,Facility Staff Count,Poverty rate,priority population(%)
0,A,98007,29174,21,10.2,14.4
1,B,98290,35990,52,6.9,14.7
2,C,98065,15548,43,1.3,14.0
3,D,98801,43291,9,11.4,18.4
4,E,98104,14144,64,23.3,15.3


In [89]:
import numpy as np

df['Population'] = df['Population'].apply(lambda x: x*0.75).apply(np.ceil)
df=df.astype({"Population": int})
df.head()


,Facility ID,Zip_Code,Population,Facility Staff Count,Poverty rate,priority population(%)
0,A,98007,21881,21,10.2,14.4
1,B,98290,26993,52,6.9,14.7
2,C,98065,11661,43,1.3,14.0
3,D,98801,32469,9,11.4,18.4
4,E,98104,10608,64,23.3,15.3


In [91]:
df.dtypes
print(df['priority population(%) '])

0    14.4
1    14.7
2    14.0
3    18.4
4    15.3
Name: priority population(%) , dtype: float64


In [93]:
df['bias']=df['Population']*(df['Poverty rate']/100)*(df['priority population(%) ']/100)
df.head()

,Facility ID,Zip_Code,Population,Facility Staff Count,Poverty rate,priority population(%),bias
0,A,98007,21881,21,10.2,14.4,321.388128
1,B,98290,26993,52,6.9,14.7,273.789999
2,C,98065,11661,43,1.3,14.0,21.223020
3,D,98801,32469,9,11.4,18.4,681.069744
4,E,98104,10608,64,23.3,15.3,378.164592


In [94]:
df['Population']=df['bias']+df['Population']
df=df.astype({"Population": int})
df=df.drop(['bias'],axis=1)
df['staff to patient ratio'] = df['Population']/ df['Facility Staff Count']
df = df.astype({"staff to patient ratio": int})
df.head()

,Facility ID,Zip_Code,Population,Facility Staff Count,Poverty rate,priority population(%),staff to patient ratio
0,A,98007,22202,21,10.2,14.4,1057
1,B,98290,27266,52,6.9,14.7,524
2,C,98065,11682,43,1.3,14.0,271
3,D,98801,33150,9,11.4,18.4,3683
4,E,98104,10986,64,23.3,15.3,171


In [97]:
# find percentage of the population of each area 
population_percentage_in_area = []
total_population = df['Population'].sum()
total_staff = df['Facility Staff Count'].sum()
for i in df['Population']:
    p = (i / total_population) * 100
    population_percentage_in_area.append(p)
    
# convert number of staff according to the population percentage in each area

staff = []
for i in df.index:
    s = (population_percentage_in_area[i] * total_staff) / 100
    staff.append(round(s))
# Display Population Percantage in each area & converted staff according to it
print(population_percentage_in_area)
print(staff)    

[21.087324050681, 25.89708033356762, 11.095492278175637, 31.485667610128605, 10.434435727447145]
[40.0, 49.0, 21.0, 60.0, 20.0]


In [100]:

new_facility_with_updated_staff = df.drop(['Facility Staff Count', 'staff to patient ratio'], axis = 1)
new_facility_with_updated_staff['New Staff Count'] = staff  
new_facility_with_updated_staff['New staff to patient ratio'] = round(new_facility_with_updated_staff['Population'] / new_facility_with_updated_staff['New Staff Count'])
new_facility_with_updated_staff

,Facility ID,Zip_Code,Population,Poverty rate,priority population(%),New Staff Count,New staff to patient ratio
0,A,98007,22202,10.2,14.4,40.0,555.0
1,B,98290,27266,6.9,14.7,49.0,556.0
2,C,98065,11682,1.3,14.0,21.0,556.0
3,D,98801,33150,11.4,18.4,60.0,552.0
4,E,98104,10986,23.3,15.3,20.0,549.0


In [102]:

# install geopy
! pip install geopy

from geopy.geocoders import Nominatim 
geolocator = Nominatim()
area = []
for i in new_facility_with_updated_staff['Zip_Code']:
    location = geolocator.geocode(i) 
    area.append(location.address)
    
area

new_facility_with_updated_staff['Area'] = area
new_facility_with_updated_staff

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


,Facility ID,Zip_Code,Population,Poverty rate,priority population(%),New Staff Count,New staff to patient ratio,Area
0,A,98007,22202,10.2,14.4,40.0,555.0,"King County, Washington, 98007, United States ..."
1,B,98290,27266,6.9,14.7,49.0,556.0,"Snohomish County, Washington, 98290, United St..."
2,C,98065,11682,1.3,14.0,21.0,556.0,"King County, Washington, 98065, United States ..."
3,D,98801,33150,11.4,18.4,60.0,552.0,"Wenatchee, Chelan County, Washington, 98801, U..."
4,E,98104,10986,23.3,15.3,20.0,549.0,"International District/Chinatown, Seattle, Kin..."


In [103]:
# arrange the columns of df
new_facility_with_updated_staff = new_facility_with_updated_staff[['Facility ID', 'Zip_Code', 'Area', 'Population', 'New Staff Count', 'New staff to patient ratio']]
new_facility_with_updated_staff

,Facility ID,Zip_Code,Area,Population,New Staff Count,New staff to patient ratio
0,A,98007,"King County, Washington, 98007, United States ...",22202,40.0,555.0
1,B,98290,"Snohomish County, Washington, 98290, United St...",27266,49.0,556.0
2,C,98065,"King County, Washington, 98065, United States ...",11682,21.0,556.0
3,D,98801,"Wenatchee, Chelan County, Washington, 98801, U...",33150,60.0,552.0
4,E,98104,"International District/Chinatown, Seattle, Kin...",10986,20.0,549.0


In [104]:
from geopy.geocoders import Nominatim 
geolocator = Nominatim()

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [108]:
# import geodesic module for finding distance between 2 locations
from geopy.distance import geodesic
iterator = len(new_facility_with_updated_staff.index)

distance_in_km = []
source_area = []
destination_area = []

# calculate distance from one area to another area and store results lists
for i in range(iterator):
    location_a = geolocator.geocode(new_facility_with_updated_staff['Zip_Code'][i]) 
    area1 = new_facility_with_updated_staff['Facility ID'][i]
    a_lat = location_a.latitude
    a_long = location_a.longitude
    point_a = (a_lat, a_long)
    
    for j in range(i+1, iterator):
        location_b = geolocator.geocode(new_facility_with_updated_staff['Zip_Code'][j]) 
        b_lat = location_b.latitude
        b_long = location_b.longitude
        point_b = (b_lat, b_long)
        distance_km = geodesic(point_a, point_b).km 
        distance_in_km.append(round(distance_km,2))
        area2 = new_facility_with_updated_staff['Facility ID'][j]
        source_area.append(area1)
        destination_area.append(area2)


In [110]:
distance_df = pd.DataFrame({"Source Area": source_area,
                           "Destination Area": destination_area,
                           "Distance (Km)": distance_in_km})

distance_df

,Source Area,Destination Area,Distance (Km)
0,A,B,36.03
1,A,C,24.20
2,A,D,137.37
3,A,E,13.93
4,B,C,48.01
5,B,D,141.32
6,B,E,41.40
7,C,D,114.18
8,C,E,37.17
9,D,E,151.04


In [115]:
distance_df_pivot = distance_df.pivot(index = "Source Area", columns = "Destination Area")
distance_df_pivot.replace(np.nan," ")

Distance (Km)                       
Destination Area             B      C       D       E
Source Area                                          
A                        36.03   24.2  137.37   13.93
B                               48.01  141.32   41.40
C                                      114.18   37.17
D                                              151.04

In [116]:

!pip install folium==0.5.0
import folium

     |████████████████████████████████| 81kB 3.7MB/s 
  Created wheel for folium: filename=folium-0.5.0-cp36-none-any.whl size=76239 sha256=a123ef3100c8117d19f6862c357a4e0a2a95d6765e3b3a82eadba00a29275d92
  Stored in directory: /root/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.5.0 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
Folium installed and imported!


In [119]:
import folium
loc = geolocator.geocode("98007") 
#print((location.latitude, location.longitude)) 
world_map = folium.Map(location=[loc.latitude, loc.longitude], zoom_start=12)

# display world map
world_map

In [123]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()
lat = loc.latitude
lng = loc.longitude
# loop through the 100 crimes and add each to the incidents feature group
incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, 
            color='black',
            fill=True,
            fill_color='red',
            fill_opacity=0.8
        )
    )

# add incidents to map
label = geolocator.geocode("98007").address
folium.Marker([lat, lng], popup=label).add_to(world_map)   
world_map.add_child(incidents)

In [139]:
latitudes = []
longitudes = []
facility_area_zip_code = ['98007','98290','98065','98801','98104'] 
facility_id = ['A','B','C','D','E']
for i in facility_area_zip_code:
    location = geolocator.geocode(i)
    lat = location.latitude
    latitudes.append(lat)
    lng = location.longitude
    longitudes.append(lng)
    #facility_id.append(facility)


In [140]:

print(facility_id)
print(facility_area_zip_code)
print(latitudes)
print(longitudes)

['A', 'B', 'C', 'D', 'E']
['98007', '98290', '98065', '98801', '98104']
[47.60983733908215, 47.93021927583956, 47.52572391933733, 47.43400063347475, 47.60070815]
[-122.14663554099312, -122.07405740297935, -121.85000267890898, -120.34113168835154, -122.33133388342861]


In [141]:
area_with_coordinates = pd.DataFrame({"Facility Id":facility_id ,
                                     "Area Zip Code": facility_area_zip_code,
                                     "Latitudes": latitudes,
                                     "Longitudes": longitudes})

area_with_coordinates

,Facility Id,Area Zip Code,Latitudes,Longitudes
0,A,98007,47.609837,-122.146636
1,B,98290,47.930219,-122.074057
2,C,98065,47.525724,-121.850003
3,D,98801,47.434001,-120.341132
4,E,98104,47.600708,-122.331334


In [162]:
# map from facility id 'A' to 'B'
import folium 

my_map1 = folium.Map(location = [area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
zoom_start = 9) 

folium.Marker([area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][0],icon=folium.Icon(color='red',icon_color='white')).add_to(my_map1) 


folium.Marker([area_with_coordinates['Latitudes'][1], area_with_coordinates['Longitudes'][1]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][1],icon=folium.Icon(color='red',icon_color='white')).add_to(my_map1) 

folium.PolyLine(locations = [(area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]), (area_with_coordinates['Latitudes'][1], area_with_coordinates['Longitudes'][1])], 
line_opacity = 0.8,line_color = 'red').add_to(my_map1) 

my_map1

In [166]:
# map from facility 'A' to 'C'
my_map2 = folium.Map(location = [area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
zoom_start = 10) 

folium.Marker([area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][0],icon=folium.Icon(color='red',icon_color='white')).add_to(my_map2) 

folium.Marker([area_with_coordinates['Latitudes'][2], area_with_coordinates['Longitudes'][2]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][2],icon=folium.Icon(color='red',icon_color='white')).add_to(my_map2) 
folium.PolyLine(locations = [(area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]), (area_with_coordinates['Latitudes'][2], area_with_coordinates['Longitudes'][2])], 
line_opacity = 0.5).add_to(my_map2) 

my_map2

In [169]:
# map from facility 'A' to 'D'
my_map3 = folium.Map(location = [area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
zoom_start = 8.3) 

folium.Marker([area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][0],icon=folium.Icon(color='red',icon_color='white')).add_to(my_map3) 

folium.Marker([area_with_coordinates['Latitudes'][3], area_with_coordinates['Longitudes'][3]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][3],icon=folium.Icon(color='red',icon_color='white')).add_to(my_map3) 
folium.PolyLine(locations = [(area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]), (area_with_coordinates['Latitudes'][3], area_with_coordinates['Longitudes'][3])], 
line_opacity = 0.5).add_to(my_map3) 

my_map3

In [170]:
# map from facility 'A' to 'E'
my_map4 = folium.Map(location = [area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
zoom_start = 9) 

folium.Marker([area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][0],icon=folium.Icon(color='red',icon_color='white')).add_to(my_map4) 

folium.Marker([area_with_coordinates['Latitudes'][4], area_with_coordinates['Longitudes'][4]], 
popup = "Facility Id: "+area_with_coordinates['Facility Id'][4],icon=folium.Icon(color='red',icon_color='white')).add_to(my_map4) 


folium.PolyLine(locations = [(area_with_coordinates['Latitudes'][0], area_with_coordinates['Longitudes'][0]), (area_with_coordinates['Latitudes'][4], area_with_coordinates['Longitudes'][4])], 
line_opacity = 0.5).add_to(my_map4) 

my_map4